In [1]:
import os 
import time
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report
from collections import Counter
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

In [2]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\1520\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
os.chdir('E:/VINOD KUMAR/Project_Guf')

In [18]:
actual_data=pd.read_csv("Guliford_cleaned_data.csv")
test_data=pd.read_csv("Guliford_test_cleaned_data.csv")

C:\Users\1520\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
actual_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86554 entries, 0 to 86553
Data columns (total 17 columns):
FileID                  86554 non-null int64
clientid                86554 non-null int64
PID                     82817 non-null float64
SUMMARY                 83731 non-null object
DATA                    86554 non-null object
Categories1             86554 non-null object
Sub_categories1         86554 non-null object
Categories2             1 non-null object
Sub_categories2         0 non-null float64
Categories3             0 non-null float64
Sub_categories3         0 non-null float64
Categories4             0 non-null float64
Sub_categories4         0 non-null float64
Categories5             0 non-null float64
Sub_categories5         0 non-null float64
Previous_Appointment    86554 non-null object
pre_data                86505 non-null object
dtypes: float64(8), int64(2), object(7)
memory usage: 11.2+ MB


In [20]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2279 entries, 0 to 2278
Data columns (total 17 columns):
FileID                  2279 non-null int64
clientid                2279 non-null int64
PID                     2279 non-null int64
SUMMARY                 2201 non-null object
DATA                    2279 non-null object
Categories1             2279 non-null object
Sub_categories1         2279 non-null object
Categories2             0 non-null float64
Sub_categories2         0 non-null float64
Categories3             0 non-null float64
Sub_categories3         0 non-null float64
Categories4             0 non-null float64
Sub_categories4         0 non-null float64
Categories5             0 non-null float64
Sub_categories5         0 non-null float64
Previous_Appointment    2279 non-null object
pre_data                2278 non-null object
dtypes: float64(8), int64(3), object(6)
memory usage: 302.8+ KB


In [22]:
actual_data=actual_data[["SUMMARY","pre_data","Categories1","Sub_categories1"]]
test_data=test_data[["SUMMARY","pre_data","Categories1","Sub_categories1"]]

In [24]:
actual_data.columns=["summary","data","categories","sub_categories"]
test_data.columns=["summary","data","categories","sub_categories"]

In [25]:
#Converting datatype of summary feature
actual_data.summary =actual_data.summary.astype('str')
actual_data.data =actual_data.data.astype('str')
actual_data.categories=actual_data.categories.astype("category")
actual_data.sub_categories=actual_data.sub_categories.astype("category")

In [26]:
test_data.summary =test_data.summary.astype('str')
test_data.data =test_data.data.astype('str')
test_data.categories=test_data.categories.astype("category")
test_data.sub_categories=test_data.sub_categories.astype("category")

In [43]:
#Some Preprocessing
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

def raw_to_prepwithtokenize( raw_review ):
    review_text = BeautifulSoup(raw_review).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    low_case = letters_only.lower()
    words = nltk.word_tokenize(low_case)
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    processed_sentence = " ".join(meaningful_words)
    return(processed_sentence)

In [46]:
length_data=len(actual_data)
length_data

86554

In [47]:
process=[]
summary_process=[]
for i in range (0,length_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    process.append(raw_to_prepwithtokenize(actual_data['data'][i]))
    summary_process.append(raw_to_prepwithtokenize(actual_data['summary'][i]))

no.of rows reviews processed: 0


C:\Users\1520\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\1520\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000
no.of rows reviews processed: 60000
no.of rows reviews processed: 70000
no.of rows reviews processed: 80000


In [48]:
length_test_data=len(test_data)

In [51]:
process_test=[]
summary_test=[]
for i in range (0,length_test_data):
    if i%1000 == 0:
        print ("no.of rows reviews processed:",i)
    process_test.append(raw_to_prepwithtokenize(test_data['data'][i]))
    summary_test.append(raw_to_prepwithtokenize(test_data['summary'][i]))

no.of rows reviews processed: 0


C:\Users\1520\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\1520\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


no.of rows reviews processed: 1000
no.of rows reviews processed: 2000


In [53]:
actual_data["data"]=pd.DataFrame(process)
actual_data["summary"]=pd.DataFrame(summary_process)
test_data["data"]=pd.DataFrame(process_test)
test_data["summary"]=pd.DataFrame(summary_test)

In [54]:
pre_data=pd.concat([actual_data["summary"],actual_data["data"]],axis=1)
target=actual_data.categories
pre_test_data = pd.concat([test_data["summary"],test_data["data"]],axis=1)
test_target=test_data.categories

In [55]:
X_train, X_test_val, Y_train, Y_test_val = train_test_split(pre_data,target, test_size=0.20,random_state=42)

In [56]:
del pre_data
del target

In [57]:
class LemmaTokenizer(object): #tokenizer for CountVectorizer for stemming using Wordnet Corpora
      def __init__(self):
            self.wnl = WordNetLemmatizer()  
      def __call__(self, doc):  
            return [self.wnl.lemmatize(t) for t in word_tokenize(doc)] 

In [58]:
#count_vectorizer= CountVectorizer(tokenizer=LemmaTokenizer(),stop_words='english',strip_accents='unicode',
#                         max_df=0.8,min_df=5,lowercase = True,ngram_range=(1,1),max_features=1000)
count_vectorizer= CountVectorizer(tokenizer=LemmaTokenizer(),stop_words='english',strip_accents='unicode',
                                  max_df=0.5,lowercase = True,ngram_range=(1,1),max_features=1000)
count_actual_data=count_vectorizer.fit_transform(X_train["data"])
count_val_data=count_vectorizer.fit_transform(X_test_val["data"])
count_test_data=count_vectorizer.fit_transform(pre_test_data["data"])

count_actual_summary=count_vectorizer.fit_transform(X_train["summary"])
count_val_summary=count_vectorizer.fit_transform(X_test_val["summary"])
count_test_summary=count_vectorizer.fit_transform(pre_test_data["summary"])

dense_actual_data=count_actual_data.todense()
dense_val_data=count_val_data.todense()
dense_test_data=count_test_data.todense()

dense_actual_summay=count_actual_summary.todense()
dense_val_summay=count_val_summary.todense()
dense_test_summary=count_test_summary.todense()


In [59]:
df_dense_actual_summay=pd.DataFrame(dense_actual_summay, columns=count_vectorizer.get_feature_names())
df_dense_val_summay=pd.DataFrame(dense_val_summay, columns=count_vectorizer.get_feature_names())
df_dense_test_summary=pd.DataFrame(dense_test_summary, columns=count_vectorizer.get_feature_names())
df_dense_actual_data=pd.DataFrame(dense_actual_data, columns=count_vectorizer.get_feature_names())
df_dense_val_data=pd.DataFrame(dense_val_data, columns=count_vectorizer.get_feature_names())
df_dense_test_data=pd.DataFrame(dense_test_data, columns=count_vectorizer.get_feature_names())

In [60]:
X_train_data=pd.concat([df_dense_actual_summay,df_dense_actual_data],axis=1)

In [61]:
X_val_data=pd.concat([df_dense_val_summay,df_dense_val_data],axis=1)

In [62]:
X_test_data=pd.concat([df_dense_test_summary,df_dense_test_data],axis=1)

In [63]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [64]:

# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_data, Y_train)

Wall time: 6.26 s


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [65]:
y_pred_train= nb.predict(X_train_data)
y_pred_val=nb.predict(X_val_data)
y_pred_test=nb.predict(X_test_data)

In [66]:
from sklearn import metrics
print(metrics.accuracy_score(Y_train, y_pred_train))
print(metrics.accuracy_score(Y_test_val, y_pred_val))
print(metrics.accuracy_score(test_target, y_pred_test))


0.680357581271753
0.4488475535786494
0.43264589732338743


In [67]:
# print the confusion matrix
print(metrics.confusion_matrix(test_target, y_pred_test))

[[ 24 277   6  17  92]
 [ 29 819   9  31  85]
 [  2  59   2   2  13]
 [ 12 215   5   9  66]
 [  7 337  13  16 132]]


In [68]:
print(classification_report(test_target, y_pred_test))

               precision    recall  f1-score   support

 APPOINTMENTS       0.32      0.06      0.10       416
 ASK_A_DOCTOR       0.48      0.84      0.61       973
          LAB       0.06      0.03      0.04        78
MISCELLANEOUS       0.12      0.03      0.05       307
 PRESCRIPTION       0.34      0.26      0.30       505

  avg / total       0.36      0.43      0.35      2279



In [69]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [70]:
# train the model using X_train_dtm
%time logreg.fit(X_train_data, Y_train)

Wall time: 32.7 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [71]:
y_log_pred_train= logreg.predict(X_train_data)
y_log_pred_val=logreg.predict(X_val_data)
y_log_pred_test=logreg.predict(X_test_data)

In [72]:
print(metrics.accuracy_score(Y_train, y_log_pred_train))
print(metrics.accuracy_score(Y_test_val, y_log_pred_val))
print(metrics.accuracy_score(test_target, y_log_pred_test))

0.8249209306355877
0.4490208537923863
0.43878894251864853


In [73]:
from sklearn.ensemble import RandomForestClassifier
model_random = RandomForestClassifier(n_estimators = 500)
model_fit = model_random.fit(X_train_data,Y_train)

In [75]:
y_rf_pred_train= model_random.predict(X_train_data)
y_rf_pred_val=model_random.predict(X_val_data)
y_rf_pred_test=model_random.predict(X_test_data)

In [76]:
print(metrics.accuracy_score(Y_train, y_rf_pred_train))
print(metrics.accuracy_score(Y_test_val, y_rf_pred_val))
print(metrics.accuracy_score(test_target, y_rf_pred_test))

0.9996389526739164
0.4716654150540119
0.37735849056603776
